# Time Series Clustering using DTW KMeans

In this notebook, we will train a KMeans Clustering algorithm based on DTW distances between Time Series data. 

We leverage the [tslearn library](https://tslearn.readthedocs.io/en/stable/index.html) for clustering. The data used in this analysis is publicly available via UCI Archive under [Online Retail II Data Set](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II).

We have cleaned and preprocessed this dataset in the optional notebook: 01. Optional - Data Cleaning and Preparation. The reader may directly use the preprocessed data included in the repository under: `./data/df_pivoted.zip` for running this notebook.

Tested with Python3, Pandas version 1.0.5.

*References*
 * Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.
 * Direct link to UCI dataset: https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx
 * tslearn github repo: https://github.com/tslearn-team/tslearn

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# optional - suppress warnings

import warnings
warnings.filterwarnings("ignore")

### Load data

In [ ]:
df_pivoted = pd.read_csv('./data/df_pivoted.zip', low_memory=False)

In [ ]:
# prepare data to laod to tslearn time_series_dataset object
df_pivoted.set_index('StockCode', inplace=True)

print(df_pivoted.shape, df_pivoted.columns)

df_pivoted.head()

## DTW KMeans Clustering

In [ ]:
%pip install tslearn

In [ ]:
from tslearn.utils import to_time_series_dataset
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans, silhouette_score

In [ ]:
%%time

# convert dataframe to time_series_dataset
X = to_time_series_dataset(df_pivoted.values)

# normalize time series to zero mean and unit variance
X_train = TimeSeriesScalerMeanVariance().fit_transform(X)

print(X.shape, X_train.shape)

In [ ]:
# create required directory structure
dir_paths = ['./tsl', './tsl/models', './tsl/plots']

for dir_path in dir_paths:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

### Perform clustering

With 12 cores, the clustering operation can take roughly half an hour. Your mileage may vary depending on your machine's configuration. `n_jobs = -1` ensures that the training uses all available cores on your machine.

Other alternative to the distance metric is "Soft-DTW" which may produce higher separation at a higher compute cost. Please see the `tslearn` documentation [link](https://tslearn.readthedocs.io/en/stable/auto_examples/clustering/plot_kmeans.html#sphx-glr-auto-examples-clustering-plot-kmeans-py) for more details.

In [ ]:
%%time

# algorithm configuration
algo = "DTW_kmeans"
metric = "dtw"

# cluster configuration
N_CLUSTERS = 3

model= TimeSeriesKMeans(n_clusters=N_CLUSTERS,
                        metric=metric,
                        n_jobs=-1,
                        random_state=0)

y_pred = model.fit_predict(X_train)

model.to_pickle(f"./tsl/models/{algo}.pkl")

In [ ]:
# backup clustering results
np.save(f"./data/tls_{algo}_cluster_labels", y_pred)

Let us plot the different clusters to visually inspect the homogeneity of the cluster composition.

In [ ]:
%%time

for yi in range(N_CLUSTERS):
    X_sub = X_train[y_pred == yi]
    ts_cnt = pd.Series(y_pred[y_pred == yi]).shape[0]
    fig = plt.figure()
    plt.title(f"{algo} | Cluster ID: {yi} | TS Count: {ts_cnt}")
    for xx in X_sub:
        plt.plot(xx.ravel(), color='xkcd:sky blue', alpha=0.025)
    fig.savefig(f"./tsl/plots/{algo}_cls_lbl_{yi}.png", dpi=150)
    plt.show()
    plt.close()

## Generate TTS for different clusters

We can now split the TTS into clusters based on the labels for the different items

In [ ]:
# convert data to TTS format expected by Forecast service
df_pivoted.reset_index(inplace=True)

df_tts = pd.melt(df_pivoted, id_vars=['StockCode'])
df_tts.columns = ['item_id', 'timestamp', 'target_value']
df_tts['timestamp'] = df_tts['timestamp'].str[:10]  # keep only the date part

print(df_tts.shape, df_tts.dtypes)

df_tts.head()

### Train - Hold-out Split

Hold-out set offers a way for verifying model performance on unseen data. With this dataset, we are looking to forecast a week out (Forecast Horizon = 1 Week) and therefore leave out a week worth of data out from the TTS as holdout set.

In [ ]:
min(df_tts['timestamp']), max(df_tts['timestamp'])

In [ ]:
df_train = df_tts[df_tts['timestamp'] < '2010-12-03']
df_test = df_tts[df_tts['timestamp'] > '2010-12-02']

df_tts.shape, df_train.shape, df_test.shape

In [ ]:
# verify that we have adequate coverage across train and test
df_tts.item_id.nunique(), df_train.item_id.nunique(), df_test.item_id.nunique()

## Split data into clusters

In [ ]:
# if restarting, reload the cluster labels
y_pred = np.load(f"./data/tls_{algo}_cluster_labels.npy")

print(y_pred)

In [ ]:
# lookup dataframe with item_ids and corresponding labels
df_lbl = pd.DataFrame()
df_lbl['item_id'] = df_pivoted['StockCode']
df_lbl['label'] = y_pred

df_lbl.shape, df_lbl.dtypes

In [ ]:
# merge labels back to the TTS
df_mrg = df_train.merge(df_lbl, how='left')

print(df_mrg.shape, df_train.shape)

In [ ]:
df_mrg.sample(5)

In [ ]:
# create required directory structure
dir_paths = ['./train', './train/cls_01', './train/cls_02', './train/cls_03']

for dir_path in dir_paths:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [ ]:
# split and save TTS
record_count = 0
for i in range(N_CLUSTERS):
    df_tmp = df_mrg[['item_id', 'timestamp', 'target_value']][df_mrg['label']==i]
    df_tmp.to_csv(f"./train/cls_0{i+1}/tts_{i+1}.csv", header=None, index=None)
    record_count += df_tmp.shape[0]
    
print(record_count, df_mrg.shape[0])  # verify that all time series are retained

### Processing Complete

These TTS files can now be uploaded to S3 and used to train Forecast models as described in the [Forecast Developers Guide](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html).